#  <font color = #025C02> Домашнее задание №3 </font>
<font color=#1A21B2>Всего будет **4 задания** по этой теме </font>

In [1]:
# Загружаем библиотеки
import numpy as np
from numpy import sqrt 
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import scikit_posthocs as sp
from math import factorial as fact
import scipy as sci
#conda install -c conda-forge scikit-posthocs
import scikit_posthocs as sp
from math import log
import math
#pip install cmh
from cmh import CMH

---
## <font color =#5D0578 >Задание №1</font>

**Рассчитайте <font color =#1A22B2 >p-value и коэффициент корреляции</font> на данных Task 1.csv:**

Ответ округлите до 4х знаков после запятой.
<font color =#1A22B2 >Какой вывод можно сделать из полученных результатов?</font>

*Примечание*: данные измерялись в ранговой шкале

---

In [2]:
# Импортируем файл с данными
task_1 = pd.read_csv('t1.csv')
print('Всего наблюдений:', len(task_1))
task_1.head(3)

Всего наблюдений: 50


,X,Y
0,94,82
1,84,102
2,122,126


In [3]:
def Evans_scale_check(r, name='r'):
    # задаем шкалу Эванса
    Evans_scale = {
        f'very weak (|{name}| < 0.19)':         0.2,
        f'weak (0.2 < |{name}| <= 0.39)':       0.4,  
        f'moderate (0.4 < |{name}| <= 0.59)':   0.6,
        f'strong (0.6 < |{name}| <= 0.79)':     0.8,
        f'very strong (0.8 < |{name}| <= 1.0)': 1.0}
    
    r_scale = list(Evans_scale.values())
    for i, elem in enumerate(r_scale):
        if abs(r) <= elem:
            conclusion_Evans_scale = list(Evans_scale.keys())[i]
            break
    return conclusion_Evans_scale    


In [4]:
# Функция для расчета и анализа корреляционного отношения
def corr_ratio_check(X, Y, p_level=0.95, orientation='XY', scale='Cheddok'):
    a_level = 1 - p_level
    X = np.array(X)
    Y = np.array(Y)
    n_X = len(X)
    n_Y = len(Y)    
    # запишем данные в DataFrame
    matrix_XY_df = pd.DataFrame({
        'X': X,
        'Y': Y})
    # число интервалов группировки
    group_int_number = lambda n: round (3.31*log(n_X, 10)+1) if round (3.31*log(n_X, 10)+1) >=2 else 2
    K_X = group_int_number(n_X)
    K_Y = group_int_number(n_Y)
    # группировка данных и формирование корреляционной таблицы
    cut_X = pd.cut(X, bins=K_X)
    cut_Y = pd.cut(Y, bins=K_Y)
    matrix_XY_df['cut_X'] = cut_X
    matrix_XY_df['cut_Y'] = cut_Y
    CorrTable_df = pd.crosstab(
        index=matrix_XY_df['cut_X'],
        columns=matrix_XY_df['cut_Y'],
        rownames=['cut_X'],
        colnames=['cut_Y'])
    CorrTable_np = np.array(CorrTable_df)
    # итоги корреляционной таблицы по строкам и столбцам
    n_group_X = [np.sum(CorrTable_np[i]) for i in range(K_X)]
    n_group_Y = [np.sum(CorrTable_np[:,j]) for j in range(K_Y)]
    # среднегрупповые значения переменной X
    Xboun_mean = [(CorrTable_df.index[i].left + CorrTable_df.index[i].right)/2 for i in range(K_X)]
    Xboun_mean[0] = (np.min(X) + CorrTable_df.index[0].right)/2    # исправляем значения в крайних интервалах
    Xboun_mean[K_X-1] = (CorrTable_df.index[K_X-1].left + np.max(X))/2
    # среднегрупповые значения переменной Y
    Yboun_mean = [(CorrTable_df.columns[j].left + CorrTable_df.columns[j].right)/2 for j in range(K_Y)]
    Yboun_mean[0] = (np.min(Y) + CorrTable_df.columns[0].right)/2    # исправляем значения в крайних интервалах
    Yboun_mean[K_Y-1] = (CorrTable_df.columns[K_Y-1].left + np.max(Y))/2
    # средневзевешенные значения X и Y для каждой группы
    Xmean_group = [np.sum(CorrTable_np[:,j] * Xboun_mean) / n_group_Y[j] for j in range(K_Y)]
    Ymean_group = [np.sum(CorrTable_np[i] * Yboun_mean) / n_group_X[i] for i in range(K_X)]
    # общая дисперсия X и Y
    Sum2_total_X = np.sum(n_group_X * (Xboun_mean - np.mean(X))**2)
    Sum2_total_Y = np.sum(n_group_Y * (Yboun_mean - np.mean(Y))**2)
    # межгрупповая дисперсия X и Y (дисперсия групповых средних)
    Sum2_between_group_X = np.sum(n_group_Y * (Xmean_group - np.mean(X))**2)
    Sum2_between_group_Y = np.sum(n_group_X * (Ymean_group - np.mean(Y))**2)
    # эмпирическое корреляционное отношение
    corr_ratio_XY = sqrt(Sum2_between_group_Y / Sum2_total_Y)
    corr_ratio_YX = sqrt(Sum2_between_group_X / Sum2_total_X)
    try:
        if orientation!='XY' and orientation!='YX':
            raise ValueError("Error! Incorrect orientation!")
        if orientation=='XY':
            corr_ratio = corr_ratio_XY
        elif orientation=='YX':
            corr_ratio = corr_ratio_YX
    except ValueError as err:
        print(err)
    # проверка гипотезы о значимости корреляционного отношения
    F_corr_ratio_calc = (n_X - K_X)/(K_X - 1) * corr_ratio**2 / (1 - corr_ratio**2)
    dfn = K_X - 1
    dfd = n_X - K_X
    F_corr_ratio_table = sci.stats.f.ppf(p_level, dfn, dfd, loc=0, scale=1)
    a_corr_ratio_calc = 1 - sci.stats.f.cdf(F_corr_ratio_calc, dfn, dfd, loc=0, scale=1)
    conclusion_corr_ratio_sign = 'significance' if F_corr_ratio_calc >= F_corr_ratio_table else 'not significance'
    # доверительный интервал корреляционного отношения
    if F_corr_ratio_calc >= F_corr_ratio_table:
        f1 = round ((K_X - 1 + n_X * corr_ratio**2)**2 / (K_X - 1 + 2 * n_X * corr_ratio**2))
        f2 = n_X - K_X
        z1 = (n_X - K_X) / n_X * corr_ratio**2 / (1 - corr_ratio**2) * 1/sci.stats.f.ppf(p_level, f1, f2, loc=0, scale=1) - (K_X - 1)/n_X
        z2 = (n_X - K_X) / n_X * corr_ratio**2 / (1 - corr_ratio**2) * 1/sci.stats.f.ppf(1 - p_level, f1, f2, loc=0, scale=1) - (K_X - 1)/n_X
        corr_ratio_conf_int_low = sqrt(z1) if sqrt(z1) >= 0 else 0
        corr_ratio_conf_int_high = sqrt(z2) if sqrt(z2) <= 1 else 1
    else:
        corr_ratio_conf_int_low = corr_ratio_conf_int_high = '-'    
    # оценка тесноты связи
    if scale=='Cheddok':
        conclusion_corr_ratio_scale = scale + ': ' + Cheddock_scale_check(corr_ratio, name=chr(951))
    elif scale=='Evans':
        conclusion_corr_ratio_scale = scale + ': ' + Evans_scale_check(corr_ratio, name=chr(951))
    # формируем результат            
    result = pd.DataFrame({
        'notation': (chr(951)),
        'coef_value': (corr_ratio),
        'coef_value_squared': (corr_ratio**2),
        'p_level': (p_level),
        'a_level': (a_level),
        'F_calc': (F_corr_ratio_calc),
        'F_table': (F_corr_ratio_table),
        'F_calc >= F_table': (F_corr_ratio_calc >= F_corr_ratio_table),
        'a_calc': (a_corr_ratio_calc),
        'a_calc <= a_level': (a_corr_ratio_calc <= a_level),
        'significance_check': (conclusion_corr_ratio_sign),
        'conf_int_low': (corr_ratio_conf_int_low),
        'conf_int_high': (corr_ratio_conf_int_high),
        'scale': (conclusion_corr_ratio_scale)
        },
        index=['Correlation ratio'])
    
    return result        

In [5]:
# Функция для проверка значимости линейной корреляционной связи
def line_corr_sign_check(X, Y, p_level=0.95, orientation='XY'):
    a_level = 1 - p_level
    X = np.array(X)
    Y = np.array(Y)
    n_X = len(X)
    n_Y = len(Y)    
    # коэффициент корреляции
    corr_coef = sci.stats.pearsonr(X, Y)[0]
    # корреляционное отношение
    try:
        if orientation!='XY' and orientation!='YX':
            raise ValueError("Error! Incorrect orientation!")
        if orientation=='XY':
            corr_ratio = corr_ratio_check(X, Y, orientation='XY', scale='Evans')['coef_value'].values[0]
        elif orientation=='YX':
            corr_ratio = corr_ratio_check(X, Y, orientation='YX', scale='Evans')['coef_value'].values[0]
    except ValueError as err:
        print(err)
    # число интервалов группировки
    group_int_number = lambda n: round (3.31*log(n_X, 10)+1) if round (3.31*log(n_X, 10)+1) >=2 else 2
    K_X = group_int_number(n_X)
    # проверка гипотезы о значимости линейной корреляционной связи
    if corr_ratio >= abs(corr_coef):
        F_line_corr_sign_calc = (n_X - K_X)/(K_X - 2) * (corr_ratio**2 - corr_coef**2) / (1 - corr_ratio**2)
        dfn = K_X - 2
        dfd = n_X - K_X
        F_line_corr_sign_table = sci.stats.f.ppf(p_level, dfn, dfd, loc=0, scale=1)
        comparison_F_calc_table = F_line_corr_sign_calc >= F_line_corr_sign_table
        a_line_corr_sign_calc = 1 - sci.stats.f.cdf(F_line_corr_sign_calc, dfn, dfd, loc=0, scale=1)
        comparison_a_calc_a_level = a_line_corr_sign_calc <= a_level
        conclusion_null_hypothesis_check = 'accepted' if F_line_corr_sign_calc < F_line_corr_sign_table else 'unaccepted'
        conclusion_line_corr_sign = 'linear' if conclusion_null_hypothesis_check == 'accepted' else 'non linear'
    else:
        F_line_corr_sign_calc = ''
        F_line_corr_sign_table = ''
        comparison_F_calc_table = ''
        a_line_corr_sign_calc = ''
        comparison_a_calc_a_level = ''
        conclusion_null_hypothesis_check = 'Attention! The correlation ratio is less than the correlation coefficient'
        conclusion_line_corr_sign = '-'
    
    # формируем результат            
    result = pd.DataFrame({
        'corr.coef.': (corr_coef),
        'corr.ratio.': (corr_ratio),
        'null hypothesis': ('r\u00b2 = ' + chr(951) + '\u00b2'),
        'p_level': (p_level),
        'a_level': (a_level),
        'F_calc': (F_line_corr_sign_calc),
        'F_table': (F_line_corr_sign_table),
        'F_calc >= F_table': (comparison_F_calc_table),
        'a_calc': (a_line_corr_sign_calc),
        'a_calc <= a_level': (comparison_a_calc_a_level),
        'null_hypothesis_check': (conclusion_null_hypothesis_check),
        'significance_line_corr_check': (conclusion_line_corr_sign),
        },
        index=['Significance of linear correlation'])
    
    return result

In [6]:
# Функция для расчета и анализа коэффициента линейной корреляции Пирсона
def corr_coef_check(X, Y, p_level=0.95, scale='Cheddok'):
    a_level = 1 - p_level
    X = np.array(X)
    Y = np.array(Y)
    n_X = len(X)
    n_Y = len(Y)
    # оценка коэффициента линейной корреляции средствами scipy
    corr_coef, a_corr_coef_calc = sci.stats.pearsonr(X, Y)
    # несмещенная оценка коэффициента линейной корреляции (при n < 15) (см.Кобзарь, с.607)
    if n_X < 15:
        corr_coef = corr_coef * (1 + (1 - corr_coef**2) / (2*(n_X-3)))
    # проверка гипотезы о значимости коэффициента корреляции
    t_corr_coef_calc = abs(corr_coef) * sqrt(n_X-2) / sqrt(1 - corr_coef**2)
    t_corr_coef_table = sci.stats.t.ppf((1 + p_level)/2 , n_X - 2)
    conclusion_corr_coef_sign = 'significance' if t_corr_coef_calc >= t_corr_coef_table else 'not significance'
    # доверительный интервал коэффициента корреляции
    if t_corr_coef_calc >= t_corr_coef_table:
        z1 = np.arctanh(corr_coef) - sci.stats.norm.ppf((1 + p_level)/2, 0, 1) / sqrt(n_X-3) - corr_coef / (2*(n_X-1))
        z2 = np.arctanh(corr_coef) + sci.stats.norm.ppf((1 + p_level)/2, 0, 1) / sqrt(n_X-3) - corr_coef / (2*(n_X-1))
        corr_coef_conf_int_low = tanh(z1)
        corr_coef_conf_int_high = tanh(z2)
    else:
        corr_coef_conf_int_low = corr_coef_conf_int_high = '-'    
    # оценка тесноты связи
    if scale=='Cheddok':
        conclusion_corr_coef_scale = scale + ': ' + Cheddock_scale_check(corr_coef)
    elif scale=='Evans':
        conclusion_corr_coef_scale = scale + ': ' + Evans_scale_check(corr_coef)
    # формируем результат            
    result = pd.DataFrame({
        'notation': ('r'),
        'coef_value': (corr_coef),
        'coef_value_squared': (corr_coef**2),
        'p_level': (p_level),
        'a_level': (a_level),
        't_calc': (t_corr_coef_calc),
        't_table': (t_corr_coef_table),
        't_calc >= t_table': (t_corr_coef_calc >= t_corr_coef_table),
        'a_calc': (a_corr_coef_calc),
        'a_calc <= a_level': (a_corr_coef_calc <= a_level),
        'significance_check': (conclusion_corr_coef_sign),
        'conf_int_low': (corr_coef_conf_int_low),
        'conf_int_high': (corr_coef_conf_int_high),
        'scale': (conclusion_corr_coef_scale)
        },
        index=['Correlation coef.'])
        
    return result 

In [7]:
# Функция для проверка значимости линейной корреляционной связи
def line_corr_sign_check(X, Y, p_level=0.95, orientation='XY'):
    a_level = 1 - p_level
    X = np.array(X)
    Y = np.array(Y)
    n_X = len(X)
    n_Y = len(Y)    
    # коэффициент корреляции
    corr_coef = sci.stats.pearsonr(X, Y)[0]
    # корреляционное отношение
    try:
        if orientation!='XY' and orientation!='YX':
            raise ValueError("Error! Incorrect orientation!")
        if orientation=='XY':
            corr_ratio = corr_ratio_check(X, Y, orientation='XY', scale='Evans')['coef_value'].values[0]
        elif orientation=='YX':
            corr_ratio = corr_ratio_check(X, Y, orientation='YX', scale='Evans')['coef_value'].values[0]
    except ValueError as err:
        print(err)
    # число интервалов группировки
    group_int_number = lambda n: round (3.31*log(n_X, 10)+1) if round (3.31*log(n_X, 10)+1) >=2 else 2
    K_X = group_int_number(n_X)
    # проверка гипотезы о значимости линейной корреляционной связи
    if corr_ratio >= abs(corr_coef):
        F_line_corr_sign_calc = (n_X - K_X)/(K_X - 2) * (corr_ratio**2 - corr_coef**2) / (1 - corr_ratio**2)
        dfn = K_X - 2
        dfd = n_X - K_X
        F_line_corr_sign_table = sci.stats.f.ppf(p_level, dfn, dfd, loc=0, scale=1)
        comparison_F_calc_table = F_line_corr_sign_calc >= F_line_corr_sign_table
        a_line_corr_sign_calc = 1 - sci.stats.f.cdf(F_line_corr_sign_calc, dfn, dfd, loc=0, scale=1)
        comparison_a_calc_a_level = a_line_corr_sign_calc <= a_level
        conclusion_null_hypothesis_check = 'accepted' if F_line_corr_sign_calc < F_line_corr_sign_table else 'unaccepted'
        conclusion_line_corr_sign = 'linear' if conclusion_null_hypothesis_check == 'accepted' else 'non linear'
    else:
        F_line_corr_sign_calc = ''
        F_line_corr_sign_table = ''
        comparison_F_calc_table = ''
        a_line_corr_sign_calc = ''
        comparison_a_calc_a_level = ''
        conclusion_null_hypothesis_check = 'Attention! The correlation ratio is less than the correlation coefficient'
        conclusion_line_corr_sign = '-'
    
    # формируем результат            
    result = pd.DataFrame({
        'corr.coef.': (corr_coef),
        'corr.ratio.': (corr_ratio),
        'null hypothesis': ('r\u00b2 = ' + chr(951) + '\u00b2'),
        'p_level': (p_level),
        'a_level': (a_level),
        'F_calc': (F_line_corr_sign_calc),
        'F_table': (F_line_corr_sign_table),
        'F_calc >= F_table': (comparison_F_calc_table),
        'a_calc': (a_line_corr_sign_calc),
        'a_calc <= a_level': (comparison_a_calc_a_level),
        'null_hypothesis_check': (conclusion_null_hypothesis_check),
        'significance_line_corr_check': (conclusion_line_corr_sign),
        },
        index=['Significance of linear correlation'])
    
    return result
line_corr_sign_check(task_1.X.to_list(), task_1.Y.to_list())

,corr.coef.,corr.ratio.,null hypothesis,p_level,a_level,F_calc,F_table,F_calc >= F_table,a_calc,a_calc <= a_level,null_hypothesis_check,significance_line_corr_check
Significance of linear correlation,0.749461,0.757694,r² = η²,0.95,0.05,0.250547,2.432236,False,0.937256,False,accepted,linear


In [8]:
print(task_1.X.corr(task_1.Y))

0.7494612835360182


In [9]:
st.ttest_ind(a=task_1.X.to_list(), b=task_1.Y.to_list())

Ttest_indResult(statistic=-1.000338027122778, pvalue=0.31961056076188976)

In [10]:
#  вероятность получить для данной вероятностной модели распределения значений случайной 
# величины такое же или более экстремальное значение статистики = 0.31961056076188976

---
## <font color =#5D0578 >Задание №2</font>

1. Постройте модель на основании данных **Task 2.csv** (доход в зависимости от группы, пола, возраста, образования)
2. Выведите отчёт о построении модели. Чему равен интерсепт полученной модели?

---

In [11]:
task_2 = pd.read_csv('t2.csv')

X = pd.get_dummies(task_2, drop_first=True)

del X['Субъект №']
y = X['Доход']
del X['Доход']
print('Всего наблюдений:', len(X))
X.head()

Всего наблюдений: 120


,Возраст,Группа_Группа 2,Пол_Мужской,Образование_Среднее,Образование_Учёная степень
0,51,0,0,1,0
1,40,1,1,0,1
2,40,1,0,0,0
3,31,1,0,1,0
4,42,1,0,1,0


In [12]:
import numpy as np
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.0018242948277267246

In [13]:
reg.intercept_

45332.26891089598

---
## <font color =#5D0578 >Задание №3</font>
Протестируйте следующую статистическую гипотезу на основании только что полученной модели

$$
H_0: \mu_{Среднее\ образование} = \mu_{Учёная\ степень}, \newline
где \mu_{i} -\ среднее\ значение\ параметра\ "Доход"\ в\ каждом\ из\ уровней\ образования
$$

Вы принимаете или отклоняете нулевую гипотезу? Почему?

---

---
## <font color =#5D0578 >Задание №4</font>
Заместите пропущенные данные из файла Task 4.csv методом MMRM.

В качестве ответа выведите <font color=blue>_№ субъекта, временную точку_</font>, в которой вы заменили данные и <font color=blue>_сам результат, округлённый до целых_

Для того, чтобы сделать это задание, нужно:

    1. Преобразовать данные из вида 
    | № субъекта | Пол     | Доход месяц 1 | ... | Доход месяц 6 |
    |------------|---------|---------------|-----|---------------|
    | 1          | Мужской | 40000         |     | 5000          |
    
    В вид
    | № субъекта | Пол     | Доход | Месяц |
    |------------|---------|-------|-------|
    | 1          | Мужской | 40000 | 1     |
    | 1          | Мужской | 50000 | 2     |
    
    2. Если есть категориальные данные, то их нужно закодировать в численные, нампример, с помощью pandas или 
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    
    Должно получиться примерно так
    
    | № субъекта | Пол     | Доход | Месяц |       | № субъекта | Пол     | Доход | Месяц |  
    |------------|---------|-------|-------|       |------------|---------|-------|-------|
    | 1          | Мужской | 40000 | 1     | -->   | 1          | 1       | 40000 | 1     |
    | ...        | ...     | ...   | ...   |       | ...        | ...     | ...   | ...   |
    | 5          | Женский | NaN   | 1     |       | 5          | 2       | NaN   | 1     |
    
    3. После этого нужно обучить модель с помощью sm.MixedLM().fit()
    4. Выделить только те значения, которые нужно заместить  предсказанными данными
    5. Заменить пропущенные значения предсказанными и отправить преподавателю
    
    Итог должен выглядеть примерно так:
    | № субъекта | Месяц | Доход |
    |------------|-------|-------|
    | 2          | 4     | 3000  |
    | 6          | 1     | 2000  |

---

In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df =pd.read_csv("t4.csv", delimiter=";")
len(df)

160

In [15]:
b = [ 'Доход месяц 2',
       'Доход месяц 3', 'Доход месяц 4', 'Доход месяц 5', 'Доход месяц 6']
g = ['Субъект №', 'Пол', 'Образование']
df1 = df[['Субъект №', 'Пол', 'Образование','Доход месяц 1']]
df1 = df1.rename(columns={'Доход месяц 1': "Доход"})
df1['m'] = 1
print(df1.columns)
for _ in b:
    l = g.copy()
    l.append(_)
    print(l)
    dft = df[l]
    dft = dft.rename(columns={_: "Доход"})
    dft['m'] = int(_[-1])
    df1 = pd.concat([df1, dft])
df1.tail()

Index(['Субъект №', 'Пол', 'Образование', 'Доход', 'm'], dtype='object')
['Субъект №', 'Пол', 'Образование', 'Доход месяц 2']
['Субъект №', 'Пол', 'Образование', 'Доход месяц 3']
['Субъект №', 'Пол', 'Образование', 'Доход месяц 4']
['Субъект №', 'Пол', 'Образование', 'Доход месяц 5']
['Субъект №', 'Пол', 'Образование', 'Доход месяц 6']


,Субъект №,Пол,Образование,Доход,m
155,156,Мужской,Среднее,40000.0,6
156,157,Мужской,Учёная степень,40000.0,6
157,158,Женский,Среднее,20000.0,6
158,159,Женский,Среднее,30000.0,6
159,160,Мужской,Высшее,40000.0,6


In [16]:
import pandas as pd
from IPython.display import display
df1['Пол'] = le.fit_transform(df1['Пол'])
df1['Образование'] = le.fit_transform(df1['Образование'])
pd.options.display.max_columns = 9
pd.options.display.max_rows = 960
display(df1)

,Субъект №,Пол,Образование,Доход,m
0,1,1,1,40000.0,1
1,2,1,1,30000.0,1
2,3,1,0,40000.0,1
3,4,1,0,30000.0,1
4,5,0,1,40000.0,1
5,6,0,1,NaN,1
6,7,1,0,30000.0,1
7,8,0,0,20000.0,1
8,9,1,0,40000.0,1
9,10,1,0,30000.0,1


In [17]:
# Загружаем библиотеки
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats

import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder


import warnings
warnings.filterwarnings("ignore")
#перед обучением необходимо удалить пропущенные значения
y = df1.dropna()['Доход'].values # зависимая переменная
ID = df1.dropna()['Субъект №'].values # идентификатор наблюдения, группирующая переменная
X = df1.dropna()[['Пол', 'Образование', 'm']].values # независимые переменные

In [18]:
#создаем и обучаем GLMM-модель
model = sm.MixedLM(y, X, groups=ID)
result_GLMM = model.fit()


In [19]:
#заменяем пропущенные значения
new_data = df1.copy()
df5 = pd.DataFrame({'Replaced':
    result_GLMM.predict(new_data.loc[new_data['Доход'].isnull(), ['Пол', 'Образование', 'm']]).round()})
df5.head()


df_to_replace = df1.loc[df1.index.isin(df5.index)]

#соединяем и оставляем только № субъекта, временную точку и замещённое значение
final = df_to_replace.merge(df5, left_index=True, right_index=True)[['Субъект №','m', 'Replaced']]

final

,Субъект №,m,Replaced
1,2,1,43875.0
1,2,2,43875.0
1,2,3,43875.0
1,2,4,43875.0
1,2,5,43875.0
1,2,6,43875.0
5,6,1,18512.0
5,6,2,18512.0
5,6,3,18512.0
5,6,4,18512.0
